In [5]:
from PyPDF2 import PdfReader
from Configs.PdfLoader import Document
documents = []
file_path="/content/Resources/thebook.pdf"
with open("/content/Resources/thebook.pdf", "rb") as f:
    reader = PdfReader(f)
    for i, page in enumerate(reader.pages):
        text = page.extract_text() or ""
        doc = Document(
            page_content=text,
            metadata={"source": file_path, "page": i + 1}
        )
        documents.append(doc)

# Example usage
print(len(documents))
print(documents[0])

234
Document(page_content='INTRODUCTION TO MACHINE LEARNING', metadata={'source': '/content/Resources/thebook.pdf', 'page': 1})


In [8]:
from Configs.RecursiveTextSplitter import RecursiveCharacterTextSplitter

# Usage
splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)


def split_documents( documents):
    """Split a list of Document objects into smaller chunk Documents."""
    chunked_documents = []
    for doc in documents:
        chunks = splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            # Copy metadata and optionally add chunk info
            metadata = dict(doc.metadata) if doc.metadata else {}
            metadata["chunk"] = i
            chunked_documents.append(Document(page_content=chunk, metadata=metadata))
    return chunked_documents


In [9]:
from PyPDF2 import PdfReader
from Configs.PdfLoader import Document
documents = []
file_path="/content/Resources/thebook.pdf"
with open("/content/Resources/thebook.pdf", "rb") as f:
    reader = PdfReader(f)
    for i, page in enumerate(reader.pages):
        text = page.extract_text() or ""
        doc = Document(
            page_content=text,
            metadata={"source": file_path, "page": i + 1}
        )
        documents.append(doc)

# Example usage
print(len(documents))
print(documents[0])

234
Document(page_content='INTRODUCTION TO MACHINE LEARNING', metadata={'source': '/content/Resources/thebook.pdf', 'page': 1})


In [ ]:

text_chunks = split_documents(documents)

print(f"Number of chunks: {len(text_chunks)}")



Number of chunks: 230


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()


/tmp/ipython-input-2103958224.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-2103958224.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hugg

In [12]:
from uuid import uuid4
from Configs.PdfLoader import Document # or your custom Document class

documents_with_id = []
for doc in text_chunks:  # your existing list of Document objects
    # add unique id to metadata and as attribute
    doc.metadata["id"] = str(uuid4())
    setattr(doc, "id", doc.metadata["id"])  # create 'id' attribute
    documents_with_id.append(doc)


In [14]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents_with_id, embeddings, collection_name="my_collection")
retriever = vectorstore.as_retriever()


In [15]:
vectorstore.similarity_search("What is this document about?",k=4)

[Document(metadata={'id': 'de5b8b4b-2bf3-478c-b038-6674c7b2692d', 'source': '/content/Resources/thebook.pdf', 'chunk': 0, 'page': 13}, page_content='1.1 A Taste of Machine Learning 5\nlanguages. In other words, we could use examples of translations to learn\nhow to translate. This machine learning approach proved quite successful\n[?].\nMany security applications, e.g. for access control, use face recognition as\none of its components. That is, given the photo (or video recording) of a\nperson, recognize who this person is. In other words, the system needs to\nclassify the faces into one of many categories (Alice, Bob, Charlie, . . . ) or\ndecide that it is an unknown face. A similar, yet conceptually quite di\x0berent\nproblem is that of veri\x0ccation. Here the goal is to verify whether the person\nin question is who he claims to be. Note that di\x0berently to before, this\nis now a yes/no question. To deal with di\x0berent lighting conditions, facial\nexpressions, whether a person i

In [27]:


# 2. Import and initialize Groq LLM
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key="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",  # Replace with your key
    model_name="llama3-8b-8192"         # Or another Groq-supported model
)

# 3. Create QA chain using retrieval and Groq LLM
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 4. Ask a question
query = "is every thing about ML in this book?"
result = qa_chain({"query": query})

# 5. Show the answer and sources
print("Answer:", result["result"])
print("\nSource Documents:")
for doc in result["source_documents"]:
    print("-", doc.metadata)
    print(doc.page_content[:300], "\n")  # Preview first 300 characters


Answer: Based on the reviews and table of contents provided, it appears that the book "Machine Learning" by Tom Mitchell (1997) covers a range of topics in machine learning, but may not include every topic in the field.

The book is described as a graduate-level textbook, and the reviews suggest that it provides a comprehensive introduction to machine learning, covering both theoretical foundations and practical applications. However, some reviewers mention that the book may be a bit dated, and that some topics may have been omitted or not fully covered.

Specifically, some reviewers mention that the book does not cover certain topics, such as:

* Statistical or data mining methods
* Boosting, Support Vector Machines, and Hidden Markov Models (which are discussed in other books or online courses)
* VC dimension (which is mentioned in one review as not being covered in the book)

On the other hand, the book appears to cover a wide range of topics in machine learning, including:

* Super